## SB-GRU-RNN + Hyperparameter GridSearch - Base Model Titik Panas

### 6. Hyperparameter GridSearchCV GRU-RNN

In [ ]:
from itertools import product
from scikeras.wrappers import KerasRegressor
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# function models
def get_model(activation='selu', optimizer='sgd', dropout_rate=0.15):

    # reset of session model
    tf.keras.backend.clear_session()

    # design network
    model = Sequential()

    # # Jika menggunakan SB-LSTM-RNN
    model.add(Bidirectional(LSTM(10, activation=activation, return_sequences=True), input_shape=(x_train.shape[1], 1)))
    model.add(Bidirectional(LSTM(10, activation=activation, return_sequences=True)))
    model.add(Bidirectional(LSTM(10, activation=activation, return_sequences=False)))
    model.add(Dropout(dropout_rate))

    # the output layer
    model.add(Dense(1))

    # compiling model
    model.compile(
        optimizer=optimizer,
        loss="mae",
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.MeanSquaredError(),
            tf.keras.metrics.MeanAbsolutePercentageError(),
        ]
    )

    # return value
    return model

In [ ]:
# Creating KerasRegressor wrapper for GridSearchCV
grid_model = KerasRegressor(model=get_model, verbose=1)

In [ ]:
# Set param for GridSearch [1]
param_grid = {
    'model__activation': ['relu', 'selu', 'elu', 'softplus'],
    'model__optimizer': ['adam', 'adamax', 'rmsprop', 'sgd'],
    'model__dropout_rate': [0.05],
    'batch_size': [2],
    'epochs': [15],
}

In [ ]:
param_grid

In [ ]:
# Menghitung jumlah kombinasi
total_combinations = len(list(product(*param_grid.values())))
print("Total kombinasi:", total_combinations)

In [ ]:
# process a gridsearchCV
grid_search = GridSearchCV(estimator=grid_model, param_grid=param_grid, n_jobs=-1, cv=2, verbose=10, scoring='neg_root_mean_squared_error')

In [ ]:
# # # from joblib import Parallel, delayed
from joblib import parallel_backend

with parallel_backend("loky"):
  grid_result = grid_search.fit(x_train, y_train)

- cek hasil gridsearchCV

In [ ]:
# summarize results
print("Best parameters: %f using %s\n" % (grid_search.best_score_, grid_search.best_params_))

In [ ]:
# show the results
result = pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),pd.DataFrame(grid_search.cv_results_["mean_test_score"], columns=["score"])],axis=1)

In [ ]:
# show the results
result = result.sort_values(by="score", ascending=False)
result

In [ ]:
# write csv form dataframe
csv_file = "M3_hasil_gridsearch.csv"
result.to_csv(csv_file, index=False)

In [ ]:
# # Specify the text file name
# csv_file = '/content/drive/MyDrive/BOPTN_2024/best_params_best_model_gru.csv'

# create dataframe from best_param
df = pd.DataFrame([grid_search.best_params_])

# write csv form dataframe
csv_file = "M3_best_param.csv"
df.to_csv(csv_file, index=False)

### 7. Evaluasi Model

In [ ]:
# Set akhir waktu komputasi
end = tm.time()

In [ ]:
# Proses menghitung waktu komputasi
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)

In [ ]:
# Hasil waktu komputasi
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))